In [ ]:
import os

import numpy as np
import pandas as pd
import featuretools as ft
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import missingno as msno

sns.set(font_scale=1.5, rc={"figure.figsize": (12, 8)})

In [ ]:
def distplot(series, **kwargs):
    """Create a figure with two subplots.
    The lower part of the figure is distplot and the upper part display
    a box plot for the same sample.

    :arg:
        series (pd.Series): The sample you want to plot.
        kwargs : all keyword argument accepted by seaborn.distplot.
    """
    # Cut the window in 2 parts
    kwrgs = {"height_ratios": (.15, .85)}
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(8, 8),
                                        gridspec_kw=kwrgs)

    # Add a graph in each part
    sns.boxplot(series, ax=ax_box)
    sns.distplot(series, ax=ax_hist, **kwargs)

    # Remove x axis name for the boxplot
    ax_box.set(xlabel='')
    return ax

## Data loading

In [ ]:
data = dict()
for dirname, _, filenames in os.walk('../data/raw'):
    for filename in filenames:
        print(filename)

**Liens entre les fichiers**

![kaggle](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)


In [ ]:
app_train = pd.read_csv('../data/raw/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
app_test = pd.read_csv('../data/raw/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

## Exploratory data analysis

### Target distribution

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
sns.countplot(app_train['TARGET'])
plt.xticks(ticks=[0, 1], labels=['Prêt remboursés', 'Défauts'])
plt.show()

### missing values

In [ ]:
incomplete = list()
for col in app_train.columns:
    if app_train[col].isna().any():
        incomplete.append(col)
        
print('the dataframe contains %i columns with missing values' % len(incomplete))

In [ ]:
msno.matrix(app_train[incomplete], labels=True)

In [ ]:
completeness = pd.DataFrame(app_train.isnull().sum(), columns=['missing values'])
completeness['% of missing'] = (completeness['missing values'] / len(app_train) * 100)
completeness.sort_values('% of missing', ascending=False)

### column types

In [ ]:
app_train.dtypes.value_counts()

### categorical variables

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique, axis=0)

### Label encoding and One-Hot Encoding

In [ ]:
app_train.info()

In [ ]:
le = LabelEncoder()
le_ = 0

for col in app_train.columns:
    if app_train[col].dtype.name == 'object':
        if len(app_train[col].unique()) <= 2:
            # label encoder
            print('Encoding %s' % col)
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            le_ += 1

print(f"{le_} columns encoded")

In [ ]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [ ]:
app_train.shape

In [ ]:
app_train.info()

In [ ]:
train_labels = app_train['TARGET']
app_train, app_test = app_train.align(app_test, join='inner', axis=1)
app_train['TARGET'] = train_labels

print(f'Train set shape : {app_train.shape}')
print(f'Test set shape : {app_test.shape}')

In [ ]:
app_train.head()

In [ ]:
app_train['TARGET']

### Anomalies

### Correlations

In [ ]:
# Find correlations with the target and sort
correlations = app_train.corr()['TARGET'].sort_values()

In [ ]:
correlations.tail(15)

In [ ]:
correlations.head(15)

In [ ]:
distplot(app_train['DAYS_BIRTH'])
plt.show()

In [ ]:
app_train['DAYS_BIRTH'] = np.abs(app_train['DAYS_BIRTH'])
sns.kdeplot(app_train[app_train['TARGET'] == 0]['DAYS_BIRTH'] / 365, label='repaid')
sns.kdeplot(app_train[app_train['TARGET'] == 1]['DAYS_BIRTH'] / 365, label='default')
plt.xlabel('Age (year)')
plt.ylabel('freq')

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
age_groups_count = age_data.groupby('TARGET')\
                        .apply(lambda x: x.groupby('YEARS_BINNED')\
                               .count()).drop(columns='TARGET').reset_index(drop=False)

In [ ]:
plt.bar(x=age_groups_count[age_groups_count['TARGET'] == 0]['YEARS_BINNED'].values.astype('str'),
        height=age_groups_count[age_groups_count['TARGET'] == 0]['DAYS_BIRTH'], label='Repaid', width=1)
plt.bar(x=age_groups_count[age_groups_count['TARGET'] == 1]['YEARS_BINNED'].values.astype('str'),
        height=age_groups_count[age_groups_count['TARGET'] == 1]['DAYS_BIRTH'], label='Default', width=1)
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize = (8, 6))

# Heatmap of correlations
sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

## Baseline : régression logistique

In [ ]:
target = app_train['TARGET']
features = list(app_train.columns)

test = app_test.copy()
train = app_train.copy().drop(columns=['TARGET'])

imputer = SimpleImputer(strategy='median')
scaler = MinMaxScaler(feature_range=(0, 1))

imputer.fit(train)

train = imputer.transform(train)
test = imputer.transform(test)

scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

print(f'train set shape : {train.shape}')
print(f'test set shape : {test.shape}')

In [ ]:
reg = LogisticRegression(C=1e-4)
reg.fit(train, target)

In [ ]:
baseline_results = app_test[['SK_ID_CURR']]
baseline_results['TARGET'] = reg.predict_proba(test)[:, 1]

In [ ]:
baseline_results.head()

In [ ]:
path = os.path.join(os.path.abspath('../reports/'), 'logistic_reg_baseline.csv')
baseline_results.to_csv(path, index=False)

### Baseline results :

**Score 0.67789**

A ce stade là, seul un fichier a été exploité. Il est quand même intéressant de voir que l'on atteind un score *ROC_AUC* de 0.68.

*ROC_AUC: Rappel* ![wiki](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/ROC_curves.svg/1280px-ROC_curves.svg.png)

## Exploration des autres fichiers

 * **bureau.csv**

    * All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
    * For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

 * **bureau_balance.csv**

    * Monthly balances of previous credits in Credit Bureau.
    * This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

 * **POS_CASH_balance.csv**

    * Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
    * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

 * **credit_card_balance.csv**

    * Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
    * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

 * **previous_application.csv**

    * All previous applications for Home Credit loans of clients who have loans in our sample.
    * There is one row for each previous application related to loans in our data sample.

    * Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
    * There is a) one row for every payment that was made plus b) one row each for missed payment.
    * One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.
 
 
 Rappel:
 
 **Liens entre les fichiers**

![kaggle](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

 

In [ ]:
bureau = pd.read_csv('../data/raw/bureau.csv')\
    .sort_values(['SK_ID_CURR', 'SK_ID_BUREAU'])\
    .reset_index(drop=True)
bureau_balance = pd.read_csv('../data/raw/bureau_balance.csv')\
    .sort_values('SK_ID_BUREAU')\
    .reset_index(drop=True)
cash = pd.read_csv('../data/raw/POS_CASH_balance.csv')\
    .sort_values(['SK_ID_CURR', 'SK_ID_PREV'])\
    .reset_index(drop=True)
credit = pd.read_csv('../data/raw/credit_card_balance.csv')\
    .sort_values(['SK_ID_CURR', 'SK_ID_PREV'])\
    .reset_index(drop=True)
previous = pd.read_csv('../data/raw/previous_application.csv')\
    .sort_values(['SK_ID_CURR', 'SK_ID_PREV'])\
    .reset_index(drop=True)
installments = pd.read_csv('../data/raw/installments_payments.csv')\
    .sort_values(['SK_ID_CURR', 'SK_ID_PREV'])\
    .reset_index(drop=True)

In [ ]:
bureau.head()

### Customers's history

In [ ]:
bureau['CREDIT_ACTIVE'].unique()

In [ ]:
bureau['CREDIT_ACTIVE'].value_counts()

In [ ]:
# distribution of the total number of past or/and actual credits per client
distplot(bureau.groupby('SK_ID_CURR').count()['SK_ID_BUREAU'])
plt.show()

In [ ]:
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], label='Closed')
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], label='Active')
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Sold'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], label='Sold')


In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes

fig, ax = plt.subplots()
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], 
            label='Closed', ax=ax)
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], 
            label='Active', ax=ax)
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Sold'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], 
            label='Sold', ax=ax)

axins = zoomed_inset_axes(ax, 5, loc='upper center')

axins.set_xlim(0, 15) # apply the x-limits
axins.set_ylim(0, 1) # apply the y-limits

sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], 
            label=None, ax=axins)
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], 
            label=None, ax=axins)
sns.kdeplot(bureau[bureau['CREDIT_ACTIVE'] == 'Sold'].groupby('SK_ID_CURR').count()['SK_ID_BUREAU'], 
            label=None, ax=axins)

plt.yticks(visible=False)
plt.xticks(visible=True)

axins.legend_.remove()

mark_inset(ax, axins, loc1=2, loc2=4, fc='none', ec='0.5')

plt.show()

In [ ]:
bureau_balance.head()

In [ ]:
bureau_balance.describe()

In [ ]:
bureau_balance[bureau_balance['SK_ID_BUREAU'] == 5001709]

In [ ]:
bureau_balance['STATUS'].value_counts()

Status of Credit Bureau loan during the month (active, closed, DPD0-30,… [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,… 5 means DPD 120+ or sold or written off ] )

DPD (Days past due)

### Récapitulons

In [ ]:
# select 5 customers from the main table
sample = app_train.sample(5, random_state=20)

In [ ]:
sample

La table principale nous renvoie les données fournies lors de la souscription.

L'un des 5 clients est un mauvais payeur (SK_ID_CURR 386051).

In [ ]:
cust_1 = bureau[bureau['SK_ID_CURR'] == 204829]
cust_1

In [ ]:
cust_2 = bureau[bureau['SK_ID_CURR'] == 386051]
cust_2

on récupère alors le passif du client fourni par les autres institutions bancaire.

In [ ]:
cust_2['SK_ID_BUREAU']

On accède finalement à la balance des crédits pour chaque crédit et pour chaque client.

In [ ]:
bureau_balance.set_index('SK_ID_BUREAU').loc[cust_1['SK_ID_BUREAU']]

In [ ]:
# bureau_balance.set_index('SK_ID_BUREAU').loc[cust_2['SK_ID_BUREAU'], :]
# KeyError: "None of [Int64Index([6194109, 6194110], dtype='int64', name='SK_ID_BUREAU')] are in the [index]"

Historique du client auprès d'home credit

In [ ]:
previous.head()

In [ ]:
previous['CODE_REJECT_REASON'].value_counts()